In [1]:
import os, json
import pandas as pd

# this finds our json files
path_to_json = 'Twitter data/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['fullname', 'text'])

i = 0
# loop over all the json files
for js in json_files:
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)

        # here need to know the layout of your json and each json has to have
        # the same structure
        index = 0
        for json_obj in json_text:
            likes = int(json_text[index]['likes'])
            replies = int(json_text[index]['replies'])
            retweets = int(json_text[index]['retweets'])
            if(likes > 0 or replies > 0 or retweets > 0):
                fullname = json_text[index]['fullname']
                text = json_text[index]['text']
                # push a list of data into a pandas DataFrame
                jsons_data.loc[i] = [fullname, text]
                i += 1
            index += 1

# now that we have the pertinent json data in our DataFrame
# print(jsons_data)


# from nltk.corpus import stopwords 
# from nltk.stem.wordnet import WordNetLemmatizer
# import string
# stop = set(stopwords.words('english'))
# exclude = set(string.punctuation) 
# lemma = WordNetLemmatizer()
# def clean(doc):
#     stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
#     punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
#     normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
#     return normalized

# doc_clean = [clean(doc).split() for doc in jsons_data['text']]

# # Importing Gensim
# import gensim
# from gensim import corpora

# # Creating the term dictionary of our courpus, where every unique term is assigned an index. 
# dictionary = corpora.Dictionary(doc_clean)

# # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
# doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

# # Creating the object for LDA model using gensim library
# Lda = gensim.models.ldamodel.LdaModel

# # Running and Trainign LDA model on the document term matrix.
# ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)


# print(ldamodel.print_topics(num_topics=5, num_words=10))


In [2]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import spacy
import re
from nltk.stem.wordnet import WordNetLemmatizer

def lemma(wordvect):    
    word = ' '.join(list(wordvect.columns))
    doc = nlp(word)
    a = wordvect.columns
    word_dict = {}
    for token in doc:
        if str(token) in a:
            word_dict[str(token)] = token.lemma_    
    word_vect = pd.DataFrame(0, index=np.arange(wordvect.shape[0]), columns=set(word_dict.values()))
    for i in word_dict.keys():
        word_vect[word_dict[i]] += wordvect[i]
    return word_vect

stop_words = ['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both', 'bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant', 'co', 'con', 'could', 'couldnt', 'cry', 'de', 'describe', 'detail', 'do', 'done', 'down', 'due', 'during', 'each', 'eg', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'found', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'hasnt', 'have', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 'hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed', 'interest', 'into', 'is', 'it', 'its', 'itself', 'keep', 'last', 'latter', 'latterly', 'least', 'less', 'ltd', 'made', 'many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 'must', 'my', 'myself', 'name', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine', 'no', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of', 'off', 'often', 'on', 'once', 'one', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'part', 'per', 'perhaps', 'please', 'put', 'rather', 're', 'same', 'see', 'seem', 'seemed', 'seeming', 'seems', 'serious', 'several', 'she', 'should', 'show', 'side', 'since', 'sincere', 'six', 'sixty', 'so', 'some', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'system', 'take', 'ten', 'than', 'that', 'the', 'their', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'thick', 'thin', 'third', 'this', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'top', 'toward', 'towards', 'twelve', 'twenty', 'two', 'un', 'under', 'until', 'up', 'upon', 'us', 'very', 'via', 'was', 'we', 'well', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with', 'within', 'without', 'would', 'yet', 'you', 'your', 'yours', 'yourself', 'yourselves', 'sonicforce', 'https', 'princesssofia', 'www', 'futurama', 'watch', 'http', 'dragonland', 'lonewolf', 'do', 'pixelworld', 'badland', 'live', 'android', 'visitor', 'city', 'hidden', 'minecraftstorymode', 'castlecat', 'minecraft', 'youtu', 'youtube', 'http', 'status', 'roblox', 'mashaandthebear', 'pokemongo', 'growtopia', 'pokemon', 'chickenrun', 'benjibanana', 'swordigo', 'pok', 'mon', 'sonicforce', 'pixelworld', 'sonic', 'castlecat', 'warlord', 'raymanadventure']



nlp = spacy.load('en')

vect = CountVectorizer(max_df = 0.2, min_df = 0.005,stop_words=stop_words,token_pattern='[a-z][a-z][a-z]+')
wordvect = pd.DataFrame(vect.fit_transform(jsons_data['text']).toarray(), columns=vect.get_feature_names())
wordvect = lemma(wordvect)  

lda = LDA(n_components=5)

a = lda.fit_transform(wordvect)
b = lda.components_
top_10_index = np.flip(np.argsort(b,axis=1),axis=1)[:,0:10]
topic = pd.DataFrame()
topic['index'] = range(1,b.shape[0]+1)
topic['top 10 words'] = ''
topic['doc_list'] = ''
for i in range(b.shape[0]):
    topic['top 10 words'].iloc[i] = list(wordvect.columns[top_10_index[i]])
    topic['doc_list'].iloc[i] = list(np.where(np.argmax(a,axis=1)==i)[0])
    
topic.to_csv("topic.csv")

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Users\Lenovo\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
